# Constructing a NeMo model

NeMo "Models" are comprised of a few key components . We'll go in the order (NN arch, Dataset + Data Loaders, Preprocessing + Postprocessing, Optimizer + scheduler , other infrastructure: tokenizers, LM configs, data augmentation, etc)

To make this slightly challenging, let's port a model from the NLP domain. Transformers are all the rage, with BERT and his friends from Sesame street forming the core infrastructure for many NLP tasks.

An excellent implementation of one such model - GPT - can be found in the `minGPT` repo https://github.com/karpathy/minGPT. We will attempt to port minGPT to NeMo 

## Construction the Neural Network Architecture

First, on the list - the neural network that forms the backbone of the NeMo model.


In [2]:
import torch
import nemo
from nemo.core import NeuralModule
from nemo.core import typecheck

[NeMo W 2023-03-21 13:13:58 optimizers:54] Apex was not found. Using the lamb or fused_adam optimizer will error out.


`NeuralModule` is a subclass of `torch.nn.Module` and it brings with it a few additional functionalities 
It has the following capabilities:
1. `Typing` it add supports for `Neural Type Checking` to the model. `Typing` is optional but quite useful
2. `Serialization` the `OmegaConf` config dict and YAML config files . all `NeuralModules` inherhently supports serialization/deserialization from such config dict
3. `FileIO` optional file serialization system

In [3]:
class MyEmptyModule(NeuralModule):

    def forward(self):
        print("Neural Module - hello world")
        

In [4]:
x = MyEmptyModule()
x()

Neural Module - hello world


## Neural types

Almost all NeMo components inherit the classs `Typing`. `Typing` is a simple class that adds 2 properties to the class that inherits it - `input_type` and `output_types`. A NeuralType is simply a semantic tensor. It contains info regarding the semantic shape and the tensor should hold, as well as the semantic info of what the tensor represents. 

So what semantic info does such a typed tensor contain? 

Across the DL domain, we often encounter cases where tensor shapes may match, but the semantics don't match at all. For example let's take a look at the following rank 3 tensors

In [8]:
# Case 1
embedding = torch.nn.Embedding(num_embeddings=10, embedding_dim=30)
x = torch.randint(high=10, size=(1,5))
print("x: ", x)
print("embedding(x) :", embedding(x).shape)

x:  tensor([[7, 9, 9, 0, 9]])
embedding(x) : torch.Size([1, 5, 30])


In [9]:
# Case 2
lstm = torch.nn.LSTM(1, 30, batch_first=True)
x = torch.randn(1, 5, 1)
print("x: ", x)
print("lstm(x) :", lstm(x)[0].shape)

x:  tensor([[[-1.6569],
         [ 0.5693],
         [-0.6563],
         [ 0.5335],
         [-0.5624]]])
lstm(x) : torch.Size([1, 5, 30])


The ability to recognize that the 2 tensors do not represent the same semantic info is precisely why we utilize Neural Types. It contains the info of both the shape and the semantic concept of what that tensor represents. If we performed a neural type check between the 2 outputs of those tensors, it would raise an error saying semantically they were different things (they are `INCOMPATIBLE` with each other)

## Neural Types - Usage

Neural Types are one of the core foundations of NeMo. While they are entirely *optional* and not instrusive, NeMo takes great care to support it so that there is no semantic incompatibility between components being used by users

In [10]:
from nemo.core.neural_types import NeuralType
from nemo.core.neural_types import *

In [11]:
class EmbeddingModule(NeuralModule):
    def __init__(self) -> None:
        super().__init__()
        self.embedding = torch.nn.Embedding(num_embeddings=10, embedding_dim=30)
    
    @typecheck()
    def forward(self, x):
        return self.embedding(x)
    
    @property
    def input_types(self):
        return {
            'x': NeuralType(axes=('B','T'), elements_type=Index())
        }

    @property
    def output_types(self):
        return {
            'y': NeuralType(axes=('B','T','C'), elements_type=EmbeddedTextType())
        }


Let's discuss how we added  type checking support to the above class
1. `forward` has a decorator `@typecheck()` on it
2. `input_types` and `output_types` properties are defined

Let's expand on each of the above steps
- `@typecheck()` is a simple decorator that takes any class that inherits `Typing` (NeuralModule does this for us) and adds the 2 default properties of `input_types` and `output_types`, which by default return None

The `@typecheck()` decorator's explicit use ensures that, by default, neural type checking is **disabled**. NeMo does not wish to intrude on the development process of models. So users can 'opt-in' to type checking by overriding the 2 properties. Therefore, the decorator ensures that users are not burdened with type checking before they wish to have it.